In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults
import re

In [2]:
tavilytool=TavilySearchResults(max_results=10)

question="how to reverse a list in python"
response=tavilytool.invoke(f"Stack overflow: {question}")
print(response)

[{'title': 'python - How do I reverse a list or loop over it backwards?', 'url': 'https://stackoverflow.com/questions/3940128/how-do-i-reverse-a-list-or-loop-over-it-backwards', 'content': 'If the goal is just to reverse the order of the items in an existing list, without looping over them or getting a copy to work with, use the <list>.reverse() function. Run this directly on a list object, and the order of all items will be reversed: [...] The generic syntax is: <object>[first_index:last_index:step]. To exploit slicing to create a simple reversed list, use: <list>[::-1]. When leaving an option empty, it sets them to defaults of the first and last element of the object (reversed if the step size is negative). [...] Put all the code snippets in this answer together to make a script that will run the different ways of reversing a list that are described below. It will time each method while running it 100,000 times. The results are shown in the last section for lists of length 2, 10, and

In [3]:
len(response)

10

In [4]:
urls=[]
qs=[]
for result in response:
    if "https://stackoverflow.com" not in result['url']:
        continue
    urls.append(result['url'])
    qs.append(result['title'])
    print(f"Title: {result['title']}")
    print(f"Link: {result['url']}")
    # print(f"Snippet: {result['snippet']}")
    print(f"Score: {result['score']}")
    print()

Title: python - How do I reverse a list or loop over it backwards?
Link: https://stackoverflow.com/questions/3940128/how-do-i-reverse-a-list-or-loop-over-it-backwards
Score: 0.8719229

Title: How could could I reverse each string in the list? - Stack Overflow
Link: https://stackoverflow.com/questions/60556085
Score: 0.6862234



In [5]:
if len(urls) > 2:
    urls = urls[:2]
def extract_question_id(url):
    # Use regex to find the numeric ID in the URL
    match = re.search(r"/questions/(\d+)", url)
    if match:
        return match.group(1)
    return None

In [6]:
q_ids=[]
for url in urls:
    q_id=extract_question_id(url)
    if q_id is not None:
        q_ids.append(q_id)
    print(f"Question ID: {q_id}")

Question ID: 3940128
Question ID: 60556085


In [7]:
id_qestion={x:y for x,y in zip(q_ids,qs)}
print(id_qestion)

{'3940128': 'python - How do I reverse a list or loop over it backwards?', '60556085': 'How could could I reverse each string in the list? - Stack Overflow'}


In [8]:
import requests
from bs4 import BeautifulSoup

# Function to beautify the HTML body
def beautify_html_body(body):
    soup = BeautifulSoup(body, "html.parser")
    return soup.get_text()

# Function to fetch answers for a question and sort them by upvotes
def get_answers_for_question(question_id):
    url = f"https://api.stackexchange.com/2.3/questions/{question_id}/answers"
    params = {
        'order': 'desc',
        'sort': 'activity',
        'site': 'stackoverflow',
        'filter': 'withbody'
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        answers = response.json().get('items', [])
        if answers:
            accepted_answer_id = response.json().get('accepted_answer_id', None)
            # Beautify body and prepare answers list
            result = []
            for answer in answers:
                is_accepted = "✅ Accepted" if answer['answer_id'] == accepted_answer_id else ""
                body = beautify_html_body(answer['body'])
                answer_info = {
                    'is_accepted': is_accepted,
                    'body': body[:300] + "..." if len(body) > 300 else body,  # Limiting body preview to 300 characters
                    'upvotes': answer['score'],
                    'link': f"https://stackoverflow.com/a/{answer['answer_id']}"
                }
                result.append(answer_info)
            # Sort answers by upvotes in descending order
            result.sort(key=lambda x: x['upvotes'], reverse=True)
            return result
        else:
            return "No answers found."
    else:
        return f"Error: {response.status_code}"


In [11]:
id_question_answers={}
for id,question in id_qestion.items():
    print(f"Fetching answers for Question ID: {id}")

    answers = get_answers_for_question(id)
    if len(answers) > 5 and isinstance(answers, list):
        answers = answers[:5]
    temp=[{"answer":answer['body'],
            "upvotes":answer['upvotes'],
            } for answer in answers]
    id_question_answers[id] = {"question":question,
                               "answers":temp}

Fetching answers for Question ID: 3940128
Fetching answers for Question ID: 60556085


In [12]:
id_question_answers

{'3940128': {'question': 'python - How do I reverse a list or loop over it backwards?',
  'answers': [{'answer': 'To get a new reversed list, apply the reversed function and collect the items into a list:\n>>> xs = [0, 10, 20, 40]\n>>> list(reversed(xs))\n[40, 20, 10, 0]\n\nTo iterate backwards through a list:\n>>> xs = [0, 10, 20, 40]\n>>> for x in reversed(xs):\n...     print(x)\n40\n20\n10\n0\n\n',
    'upvotes': 1718},
   {'answer': '>>> xs = [0, 10, 20, 40]\n>>> xs[::-1]\n[40, 20, 10, 0]\n\nExtended slice syntax is explained here. See also, documentation.\n',
    'upvotes': 1542},
   {'answer': 'Use list.reverse to reverse a list in-place:\n>>> xs = [0, 10, 20, 40]\n>>> xs.reverse()\n>>> xs\n[40, 20, 10, 0]\n\nUse slices to create a new list with the items in reverse order:\n>>> xs[::-1]\n[40, 20, 10, 0]\n\n',
    'upvotes': 473},
   {'answer': 'Summary of Reverse Methods\nThere are three different built-in ways to reverse a list. Which method is best depends on whether you need t